<a href="https://colab.research.google.com/github/saima8/Deep-Learning/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import string

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize

In [ ]:
print('Current Directory: ', os.getcwd())
print('Directories in Parent Directory: ', os.listdir('../'))

Current Directory:  /kaggle/working
Directories in Parent Directory:  ['lib', 'input', 'working']


In [ ]:
input_directory = '../input'
print('Directories in Input Directory: ', os.listdir(input_directory))

Directories in Input Directory:  ['asl-alphabet']


In [ ]:
asl_directory = '../input/asl-alphabet'
train_directory = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_directory = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [ ]:
class ASLDatasetTest(Dataset):
    char_to_int = {c: ord(c) - ord('A') for c in string.ascii_uppercase}
    char_to_int['del'] = 26
    char_to_int['nothing'] = 27
    char_to_int['space'] = 28
    int_to_char = {value: key for key, value in char_to_int.items()}
        
    def __init__(self, directory: str, transform=None, label_transform=None):
        super().__init__()        
        self.directory = directory
        self.transform = transform
        self.label_transform = label_transform        
        self.x = None
        self.y = None
        self._load_images()
    
    def __getitem__(self, idx):
        x, y = torchvision.io.read_image(self.x[idx]).type(torch.float32), self.y[idx]
        if self.transform:
            x = self.transform(x)
        if self.label_transform:
            y = self.label_transform(y)
        return x, y
    
    def __len__(self):
        return len(self.y)
    
    def _load_images(self):
        self.x = []
        self.y = []
        for img in os.listdir(self.directory):
            class_name = img[:1]
            if 'nothing' in img:
                class_name = 'nothing'
            elif 'space' in img:
                class_name = 'space'
            elif 'del' in img:
                class_name = 'del'
            self.x.append(os.path.join(self.directory, img))
            self.y.append(self.char_to_int[class_name])
        self.y = torch.tensor(self.y, dtype=torch.int64)
    
    @staticmethod
    def get_classname(idx: int) -> str:
        return ASLDataset.int_to_char[idx]

In [ ]:
class ASLDataset(Dataset):
    char_to_int = {c: ord(c) - ord('A') for c in string.ascii_uppercase}
    char_to_int['del'] = 26
    char_to_int['nothing'] = 27
    char_to_int['space'] = 28
    int_to_char = {value: key for key, value in char_to_int.items()}
        
    def __init__(self, directory: str, transform=None, label_transform=None):
        super().__init__()
        
        self.directory = directory
        self.transform = transform
        self.label_transform = label_transform
        
        self.x = None
        self.y = None
        self._load_images()
    
    def __getitem__(self, idx):
        x, y = torchvision.io.read_image(self.x[idx]).type(torch.float32), self.y[idx]
        
        if self.transform:
            x = self.transform(x)
        if self.label_transform:
            y = self.label_transform(y)
        
        return x, y
    
    def __len__(self):
        return len(self.y)
    
    def _load_images(self):
        self.x = []
        self.y = []
        
        for c in os.listdir(self.directory):
            class_name = c
            class_dir = os.path.join(self.directory, class_name)
            for img in os.listdir(class_dir):
                self.x.append(os.path.join(class_dir, img))
                self.y.append(self.char_to_int[class_name])
                
        self.y = torch.tensor(self.y, dtype=torch.int64)
    
    @staticmethod
    def get_classname(idx: int) -> str:
        return ASLDataset.int_to_char[idx]

In [ ]:
ts = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
asl_train = ASLDataset('../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train', transform=ts)
asl_test = ASLDatasetTest('../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test', transform=ts)